In [28]:
# import pandas as pd
import numpy as np
import pandas as pd
# Đường dẫn đến tệp dữ liệu trên Kaggle
file_path = '/kaggle/input/vindr-mammo-dataset/breast-level_annotations.csv'
pairs = []


# Load 100 mẫu đầu tiên từ tệp CSV vào một DataFrame
df = pd.read_csv(file_path).head(5000)

df.loc[df['breast_birads'] == 'BI-RADS 1', 'breast_birads'] = '1'
df.loc[df['breast_birads'] == 'BI-RADS 2', 'breast_birads'] = '2'
df.loc[df['breast_birads'] == 'BI-RADS 3', 'breast_birads'] = '3'
df.loc[df['breast_birads'] == 'BI-RADS 4', 'breast_birads'] = '4'
df.loc[df['breast_birads'] == 'BI-RADS 5', 'breast_birads'] = '5'

df_new = df[['image_id', 'breast_birads']].copy()

groups = df_new.groupby('breast_birads')


# Lấy mỗi nhóm một phần tử và kết hợp chúng lại với nhau
result = pd.concat([group.head(10).reset_index(drop=True) for name, group in groups])

new_df = pd.DataFrame(columns=result.columns)
selected_rows_list = []

for i in range(0, 10):
    selected_rows = result.loc[result.index == i]
    selected_rows_list.append(selected_rows)

    # Hiển thị kết quả
new_df = pd.concat(selected_rows_list)

# Print the new DataFrame
print(new_df)



                           image_id breast_birads
0  dd9ce3288c0773e006a294188aadba8e             1
0  d8125545210c08e1b1793a5af6458ee2             2
0  aad38467d1f04cedf300c9c8305a3536             3
0  2b46a6fc2c076cfc187b385f6a6f40e6             4
0  581b0e4d471a5c0adc6888cb038fa722             5
1  57fbdd278af5c8789a02b355c11620d4             1
1  290c658f4e75a3f83ec78a847414297c             2
1  2ccefd9e7701acf52c92275e844f7578             3
1  8338207743a394ab8b56b5b920fc30b6             4
1  29896919b15b2dd978a36b2f68a5ad93             5
2  202d761a6b0f86faaeefc39ee18b1c53             1
2  cd0fc7bc53ac632a11643ac4cc91002a             2
2  39a9809abe1441f60f9fab50679c2a73             3
2  396bea28fd4be11598bc53bf5f2b7460             4
2  5d13674e2ed04208bf232e2eeb468f7a             5
3  acccc1727b61b261844d86aa9de53536             1
3  71638b1e853799f227492bfb08a01491             2
3  6770e08049264165b58c5f5ddcfd5b95             3
3  df4b1cfb384dab76abc34cc4db0f5b30             4


In [29]:
import cv2
import pandas as pd
import os
import torch
from torchvision import transforms

In [30]:
from PIL import Image
from torchvision import transforms

def load_image(image_path):
    # Read the image from the path
    image = cv2.imread(image_path)
    
    # Convert the image from OpenCV's BGR format to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Convert the NumPy array to a PIL image
    pil_image = Image.fromarray(image)
    
    # Define the transformation pipeline
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the image
        transforms.ToTensor()  # Transform the image to a tensor
    ])
    
    # Apply the transformations to the image
    image_tensor = transform(pil_image)
    
    return image_tensor



# Thư mục chứa ảnh
image_folder = "/kaggle/input/vindr-mammo-dataset/Processed_Images"

# Đọc DataFrame từ file CSV
df = new_df
print(df)

# Tạo một cột mới trong DataFrame để lưu tensor của ảnh
df['image_tensor'] = None

# Hàm đệ quy để tìm ảnh trong tất cả các thư mục con
def find_images(image_id, folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file == image_id+'.png':
                image_paths=os.path.join(root, file)
    return image_paths



                           image_id breast_birads
0  dd9ce3288c0773e006a294188aadba8e             1
0  d8125545210c08e1b1793a5af6458ee2             2
0  aad38467d1f04cedf300c9c8305a3536             3
0  2b46a6fc2c076cfc187b385f6a6f40e6             4
0  581b0e4d471a5c0adc6888cb038fa722             5
1  57fbdd278af5c8789a02b355c11620d4             1
1  290c658f4e75a3f83ec78a847414297c             2
1  2ccefd9e7701acf52c92275e844f7578             3
1  8338207743a394ab8b56b5b920fc30b6             4
1  29896919b15b2dd978a36b2f68a5ad93             5
2  202d761a6b0f86faaeefc39ee18b1c53             1
2  cd0fc7bc53ac632a11643ac4cc91002a             2
2  39a9809abe1441f60f9fab50679c2a73             3
2  396bea28fd4be11598bc53bf5f2b7460             4
2  5d13674e2ed04208bf232e2eeb468f7a             5
3  acccc1727b61b261844d86aa9de53536             1
3  71638b1e853799f227492bfb08a01491             2
3  6770e08049264165b58c5f5ddcfd5b95             3
3  df4b1cfb384dab76abc34cc4db0f5b30             4


In [31]:

image_paths=[]
for index, row in df.iterrows():
    image_id = row['image_id']
    paths = find_images(image_id, image_folder)
    image_paths.append(paths)
    image_tensor = load_image(paths)
    #df['image_tensor'] = image_tensor

   


In [32]:
pip install vit-pytorch


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install timm

Note: you may need to restart the kernel to use updated packages.


In [34]:
import torch
from torchvision.models import resnet50, resnet152, densenet121
from vit_pytorch import ViT
import timm
import torch

def load_model(model_type, checkpoint_path):
    if model_type == 'resnet50':
        model = resnet50(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = torch.nn.Linear(num_ftrs, 512)
    elif model_type == 'resnet152':
        model = resnet152(pretrained=False)
    elif model_type == 'densenet121':
        model = densenet121(pretrained=True)
        num_classes = 512
        model.classifier = torch.nn.Linear(model.classifier.in_features, num_classes)
    elif model_type == 'vit':
        model_name = 'vit_base_patch16_224'  # Specify the model name
        num_classes = 512  # Number of output classes (adjust as needed)
        model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)

    else:
        raise ValueError(f"Loại mô hình '{model_type}' không được hỗ trợ.")
    
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    
    # Extract the model state dictionary from the checkpoint
    if 'model_state_dict' in checkpoint:
        state_dict = checkpoint['model_state_dict']
    else:
        state_dict = checkpoint  # Assuming the entire checkpoint is the state_dict
    
    # Adjust state dictionary keys if necessary
    new_state_dict = {k.replace('model.', ''): v for k, v in state_dict.items()}
    model.load_state_dict(new_state_dict)
    return model
    
    # Load the state


In [35]:
checkpoint='/kaggle/input/ver1-checkpoint/version 1/weights_setting3/vit_base_patch16_224BasedModel/best.pt'
model=load_model('vit',checkpoint)


In [1]:
import numpy as np

def vector_length(v):
    return np.linalg.norm(v)
def euclidean_distance(vector1, vector2):
    """
    Tính Euclidean distance giữa hai vector trong không gian nhiều chiều.
​
    Tham số:
    vector1, vector2: iterable, hai vector trong không gian nhiều chiều.
​
    Trả về:
    distance: float, Euclidean distance giữa hai vector.
    """
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)
    vector3= vector1-vector2
    # Tính Euclidean distance
    distance = vector_length(vector3)
    
    return distance


# Ví dụ về vector cần tính độ dài
vector = np.array([1, 2, 3])

# Tính độ dài của vector
length = vector_length(vector)

print("Độ dài của vector:", length)


Độ dài của vector: 3.7416573867739413


In [2]:
import numpy as np
from scipy.optimize import minimize

def estimate_theta_from_cosine_similarity(C):
    """
    Tính toán các tham số theta từ ma trận cosine similarity sử dụng mô hình Bradley-Terry.

    Args:
    C (np.ndarray): Ma trận cosine similarity với kích thước n x n.

    Returns:
    tuple: Ma trận đối đầu Bradley-Terry và mảng các tham số theta ước lượng.
    """
    def cosine_to_bradley_terry_matrix(C):
        """
        Chuyển đổi ma trận cosine similarity thành ma trận đối đầu Bradley-Terry.
        
        Args:
        C (np.ndarray): Ma trận cosine similarity với kích thước n x n.

        Returns:
        np.ndarray: Ma trận đối đầu Bradley-Terry với kích thước n x n.
        """
        n = C.shape[0]
        W = np.zeros((n, n))

        for i in range(n-1):
            for j in range(i+1, n): # Bắt đầu từ i+1 để tránh so sánh với chính nó và tránh lặp lại các so sánh
                if i == n-1:  # Nếu i là ảnh cuối cùng
                    W[i, j] = 1
                    break
                if C[i, i+1] >= C[i, j]:  # So sánh với giá trị C[i, i+1] thay vì với C[i, j]
                    W[i, i+1] = 1  # Gán giá trị 1 cho W[i, i+1] khi i thắng i+1

        return W

    def log_likelihood(theta, W):
        """
        Hàm tính log-likelihood cho mô hình Bradley-Terry.
        
        Args:
        theta (np.ndarray): Mảng các tham số theta.
        W (np.ndarray): Ma trận đối đầu Bradley-Terry.

        Returns:
        float: Giá trị âm của log-likelihood.
        """
        n = len(theta)
        ll = 0
        for i in range(n):
            for j in range(n):
                if i != j:
                    ll += W[i, j] * np.log(theta[i] / (theta[i] + theta[j]))
        return -ll

    # Chuyển đổi ma trận cosine similarity thành ma trận đối đầu
    W = cosine_to_bradley_terry_matrix(C)

    # Khởi tạo giá trị ban đầu cho theta
    initial_theta = np.ones(W.shape[0])

    # Tối ưu hóa log-likelihood
    result = minimize(log_likelihood, initial_theta, args=(W,), method='L-BFGS-B', bounds=[(1e-5, None)] * len(initial_theta))
    estimated_theta = result.x

    return W, estimated_theta

# Ví dụ sử dụng hàm với ma trận cosine similarity
C = np.array([
    [0, 0.8, 0.6, 0.4, 0.2],
    [0, 0, 0.2, 0.5, 0.3],
    [0, 0, 0, 0.9, 0.5],
    [0, 0, 0, 0, 0.6],
    [0, 0, 0, 0, 0]
])

W, estimated_theta = estimate_theta_from_cosine_similarity(C)
print("Ma trận đối đầu W:")
print(W)
print("Estimated theta:", estimated_theta)


Ma trận đối đầu W:
[[0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]]
Estimated theta: [6.60851098e+04 4.19317541e+02 1.91817150e+00 4.37917822e-03
 1.00000000e-05]


In [63]:
import numpy as np
import torch.nn.functional as F
def savee_matrix(df,model):
    n = 5
    for i in range(0, 10):
        output = []
        matrix = np.zeros((n, n))  
        selected_rows = result.loc[result.index == i]
        for index, row in selected_rows.iterrows():
            image_id = row['image_id']
            paths = find_images(image_id, image_folder)
            input_tensor = load_image(paths)
            input_batch = input_tensor.unsqueeze(0)

            if torch.cuda.is_available():
                input_batch = input_batch.to('cuda')
                model.to('cuda')

            with torch.no_grad():
                out = model(input_batch)
                output.append(out)
        for i in range(0, 5):
            for j in range(0, 5):
                if i == j:
                    matrix[i, j]=0
                else:
                    matrix[i, j] = F.cosine_similarity(output[i], output[j], dim=1)
                    matrix[j, i] = matrix[i, j]  # Since it's symmetric, fill the corresponding element
        print(matrix)
        W, estimated_theta = estimate_theta_from_cosine_similarity(matrix)
        print("Ma trận đối đầu W:")
        print(W)
        print("Estimated theta:", estimated_theta)

        
    return 


In [65]:
savee_matrix(df,model)

[[ 0.          0.84463805 -0.00099976  0.38868359  0.50541222]
 [ 0.84463805  0.         -0.08234338  0.25645661  0.54351336]
 [-0.00099976 -0.08234338  0.          0.49049935 -0.17193416]
 [ 0.38868359  0.25645661  0.49049935  0.         -0.02795375]
 [ 0.50541222  0.54351336 -0.17193416 -0.02795375  0.        ]]
Ma trận đối đầu W:
[[0. 1. 1. 1. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]]
Estimated theta: [4.20824457e+03 6.31372752e+01 4.96566780e-01 2.23330295e-03
 1.00000000e-05]
[[ 0.          0.79716074  0.70120782  0.57289755 -0.0241006 ]
 [ 0.79716074  0.          0.77283013  0.60819769  0.10276987]
 [ 0.70120782  0.77283013  0.          0.68917781  0.33812472]
 [ 0.57289755  0.60819769  0.68917781  0.          0.27639854]
 [-0.0241006   0.10276987  0.33812472  0.27639854  0.        ]]
Ma trận đối đầu W:
[[0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]]
Estimated theta: [1.12237724e+03 1.95509022e+01 4.140

In [49]:
import numpy as np

# Hàm tính DCG
def calculate_dcg(ranking):
    return np.sum(ranking / np.log2(np.arange(2, len(ranking) + 2)))

# Tính NDCG cho mỗi hàng của ma trận xếp hạng
def calculate_ndcg(cosine_similarity_matrix, ideal_ranking):
    ndcg_scores = []
    for i in range(cosine_similarity_matrix.shape[0]):
        # Sắp xếp xếp hạng theo thứ tự giảm dần của cosine similarity
        sorted_indices = np.argsort(-cosine_similarity_matrix[i])
        
        sorted_ranking = ideal_ranking[i][sorted_indices]

        # Tính toán DCG
        dcg = calculate_dcg(sorted_ranking)

        # Tính toán IDCG (với xếp hạng tốt nhất)
        ideal_sorted_ranking = np.sort(ideal_ranking[i])[::-1]
        idcg = calculate_dcg(ideal_sorted_ranking)

        # Tính toán NDCG
        if idcg == 0:
            ndcg = 0
        else:
            ndcg = dcg / idcg
        ndcg_scores.append(ndcg)
    return np.array(ndcg_scores)

# Ideal ranking matrix
ideal_ranking = np.array([[4, 3, 2, 1, 0],
                          [3, 4, 2, 0, 1],
                          [2, 2, 4, 2, 2],
                          [1, 0, 2, 4, 3],
                          [0, 1, 2, 3, 4]])



In [50]:
import numpy as np
import torch.nn.functional as F
def save_matrix(df,model):
    n = 5
    matrix_root = np.zeros((n, n)) 
    matrix_root_tensor = torch.tensor(matrix_root)
    for i in range(0, 10):
        output = []
        matrix = np.zeros((n, n))  
        selected_rows = result.loc[result.index == i]
        for index, row in selected_rows.iterrows():
            image_id = row['image_id']
            paths = find_images(image_id, image_folder)
            print(image_id,row['breast_birads'])
            input_tensor = load_image(paths)
            input_batch = input_tensor.unsqueeze(0)

            if torch.cuda.is_available():
                input_batch = input_batch.to('cuda')
                model.to('cuda')

            with torch.no_grad():
                out = model(input_batch)
                output.append(out)
        for i in range(0, 5):
            for j in range(0, 5):
                if i == j:
                    matrix[i, j]=1
                else:
                    matrix[i, j] = F.cosine_similarity(output[i], output[j], dim=1)
                    matrix[j, i] = matrix[i, j]  # Since it's symmetric, fill the corresponding element
        print(matrix)
        ndcg_scores = calculate_ndcg(matrix, ideal_ranking)
        print(ndcg_scores)
         
        
    return 


trả về NDCG score của mỗi hàng. cách đánh giá NDCG score: 

Gần 1 (0.9 - 1): Đây là kết quả xuất sắc, cho thấy hệ thống xếp hạng gần như hoàn hảo trong việc xếp hạng các kết quả theo độ liên quan.

0.7 - 0.9: Đây là kết quả tốt, cho thấy hệ thống xếp hạng hoạt động khá hiệu quả, mặc dù vẫn còn chỗ để cải thiện.

0.5 - 0.7: Đây là kết quả trung bình, cho thấy hệ thống có thể chấp nhận được nhưng cần cải thiện để đạt hiệu suất tốt hơn.

Dưới 0.5: Đây là kết quả thấp, cho thấy hệ thống xếp hạng hoạt động không tốt và cần có sự điều chỉnh đáng kể.


In [51]:
save_matrix(df,model)

dd9ce3288c0773e006a294188aadba8e 1
d8125545210c08e1b1793a5af6458ee2 2
aad38467d1f04cedf300c9c8305a3536 3
2b46a6fc2c076cfc187b385f6a6f40e6 4
581b0e4d471a5c0adc6888cb038fa722 5
[[ 1.00000000e+00  8.44638050e-01 -9.99756623e-04  3.88683587e-01
   5.05412221e-01]
 [ 8.44638050e-01  1.00000000e+00 -8.23433772e-02  2.56456614e-01
   5.43513358e-01]
 [-9.99756623e-04 -8.23433772e-02  1.00000000e+00  4.90499347e-01
  -1.71934158e-01]
 [ 3.88683587e-01  2.56456614e-01  4.90499347e-01  1.00000000e+00
  -2.79537514e-02]
 [ 5.05412221e-01  5.43513358e-01 -1.71934158e-01 -2.79537514e-02
   1.00000000e+00]]
[0.9691001  0.97856603 1.         0.94523797 0.914412  ]
57fbdd278af5c8789a02b355c11620d4 1
290c658f4e75a3f83ec78a847414297c 2
2ccefd9e7701acf52c92275e844f7578 3
8338207743a394ab8b56b5b920fc30b6 4
29896919b15b2dd978a36b2f68a5ad93 5
[[ 1.          0.79716074  0.70120782  0.57289755 -0.0241006 ]
 [ 0.79716074  1.          0.77283013  0.60819769  0.10276987]
 [ 0.70120782  0.77283013  1.          0.